In [2]:
!pip install grpcio==1.60.0 | tail -n 1
!pip install pymilvus | tail -n 1

In [3]:
!pip install ipython-sql==0.4.1 | tail -n 1
!pip install sqlalchemy==1.4.46 | tail -n 1
!pip install sqlalchemy==1.4.46 "pyhive[presto]" | tail -n 1
!pip install sentence_transformers | tail -n 1
!pip install python-dotenv | tail -n 1
!pip install ibm-cloud-sdk-core | tail -n 1

In [17]:
!pip install ibm_watson_machine_learning | tail -n1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
E0507 15:35:39.134888000 13732294656 ssl_transport_security.cc:1511]   Handshake failed with fatal error SSL_ERROR_SSL: error:1000007d:SSL routines:OPENSSL_internal:CERTIFICATE_VERIFY_FAILED.


E0507 15:35:48.185981000 13840625664 ssl_transport_security.cc:1511]   Handshake failed with fatal error SSL_ERROR_SSL: error:1000007d:SSL routines:OPENSSL_internal:CERTIFICATE_VERIFY_FAILED.


In [4]:
PROJECT_ID="0f894bac-9abe-4a26-a17c-958186743029"
ACCESS_TOKEN="p-2+l6q+zqZH5sLma42mP5oIaw==;7GofcXPpkilxSWLnqpRkIQ==:N7ooxvSB8bH0NjnAMTt9+5tTm7F6AJIFiloNFkWc0CVyTZWoyym2RFQTctDmh0frrgOAr1e7m28YnQj4/GLgcltCsXNX5vdgOA=="

In [11]:
import os
from dotenv import load_dotenv
from ibm_cloud_sdk_core import IAMTokenManager
import warnings
warnings.filterwarnings('ignore')

load_dotenv('config.env')

# Connection variables
api_key = os.getenv("API_KEY", None)
ibm_cloud_url = os.getenv("IBM_CLOUD_URL", None) 
project_id = os.getenv("PROJECT_ID", None)

creds = {
    "url": ibm_cloud_url,
    "apikey": api_key 
}
access_token = IAMTokenManager(
    apikey = api_key,
    url = "https://iam.cloud.ibm.com/identity/token"
).get_token()

In [12]:
from sentence_transformers import SentenceTransformer
from pymilvus import(
    Milvus,
    IndexType,
    Status,
    connections,
    FieldSchema,
    DataType,
    Collection,
    CollectionSchema,
)

import os 

host = os.getenv("MILVUS_HOST", None)
port = os.getenv("MILVUS_PORT", None)
password = 'password'
user = 'ibmlhadmin'
server_pem_path = 'cert.crt'

connections.connect(alias = 'default',
                   host = host,
                   port = port,
                   user = user,
                   password = password,
                   server_pem_path = server_pem_path,
                   server_name = 'watsonxdata',
                   secure = True)

# Load collection

basic_collection = Collection("wiki_articles")      
basic_collection.load()

# Query function
def query_milvus(query, num_results=5):
    
    # Vectorize query
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2') # 384 dim
    query_embeddings = model.encode([query])

    # Search
    search_params = {
        "metric_type": "L2", 
        "params": {"nprobe": 5}
    }
    results = basic_collection.search(
        data=query_embeddings, 
        anns_field="vector", 
        param=search_params,
        limit=num_results,
        expr=None, 
        output_fields=['article_text'],
    )
    return results

In [13]:
## Consider how climate change may relate to other industries and processes related to your business

question_text = "What can my company do to help fight climate change?"

E0507 15:33:50.020963000 13732294656 ssl_transport_security.cc:1511]   Handshake failed with fatal error SSL_ERROR_SSL: error:1000007d:SSL routines:OPENSSL_internal:CERTIFICATE_VERIFY_FAILED.


In [31]:
# Query Milvus 

def get_chunks(query):
    num_results = 3
    results = query_milvus(query, num_results)

    relevant_chunks = []
    for i in range(num_results):    
        #print(f"id: {results[0].ids[i]}")
        #print(f"distance: {results[0].distances[i]}")
        text = results[0][i].entity.get('article_text')
        relevant_chunks.append(text)
    
    #print(relevant_chunks)
    return relevant_chunks

E0507 16:19:34.998575000 13732294656 ssl_transport_security.cc:1511]   Handshake failed with fatal error SSL_ERROR_SSL: error:1000007d:SSL routines:OPENSSL_internal:CERTIFICATE_VERIFY_FAILED.
E0507 16:19:59.153999000 13840625664 ssl_transport_security.cc:1511]   Handshake failed with fatal error SSL_ERROR_SSL: error:1000007d:SSL routines:OPENSSL_internal:CERTIFICATE_VERIFY_FAILED.


In [32]:
def make_prompt(context, question_text):
    context = "\n\n".join(context)
    return (f"{context}\n\nPlease answer a question using this text. "
          + f"If the question is unanswerable, say \"unanswerable\"."
          + f"\n\nQuestion: {question_text}")


In [19]:
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

# Model Parameters
params = {
        GenParams.DECODING_METHOD: "greedy",
        GenParams.MIN_NEW_TOKENS: 1,
        GenParams.MAX_NEW_TOKENS: 500,
        GenParams.TEMPERATURE: 0,
}
model = Model(
        model_id='meta-llama/llama-2-70b-chat', 
        params=params, credentials=creds, 
        project_id=project_id
)

# Prompt LLM
def ask_llm(prompt):
        response = model.generate_text(prompt)
        print(f"Question: {question_text}{response}")
        return response

E0507 15:49:55.819932000 13732294656 ssl_transport_security.cc:1511]   Handshake failed with fatal error SSL_ERROR_SSL: error:1000007d:SSL routines:OPENSSL_internal:CERTIFICATE_VERIFY_FAILED.


In [36]:
from ipywidgets import widgets

text_input = widgets.Text(value=question_text, disabled=False)
result_text = widgets.Textarea(value='', disabled=True)

def on_click(b):
#    display('You clicked the button!' + text_input.value + " radio=" + radio.value)
    if (radio.value == "with RAG"):
        result_text.value = "asking AI (with RAG) ..."
        prompt = text_input.value
    else:
        result_text.value = "asking AI (without RAG) ..."
        chunks = get_chunks(text_input.value)
        prompt = make_prompt(chunks, text_input.value)
    result_text.value = ask_llm(prompt)

button = widgets.Button(description='Ask AI');
button.on_click(on_click)

radio=widgets.RadioButtons(options=['without RAG', 'with RAG'], description='Please choose')

radio_box  = widgets.HBox([radio])
input_box  = widgets.Box([widgets.Label('Your question!'), text_input, button])
result_box = widgets.Box([result_text],
                         layout=widgets.Layout(height='300px', width='1000px'))
box = widgets.Box(children=[radio_box, input_box , result_box],
                  layout=widgets.Layout(display='flex',
                  flex_flow='wrap',
                  justify_content='space-between',
                  align_items='center',
                  width='100%'))

display(box)


Box(children=(HBox(children=(RadioButtons(description='Please choose', options=('without RAG', 'with RAG'), va…

Question: What can my company do to help fight climate change?

Answer: Your company can reduce its carbon footprint by transitioning to renewable energy sources, implementing energy-efficient technologies, reducing waste and emissions, and promoting sustainable practices throughout its supply chain. Additionally, it can invest in carbon offsetting projects, such as reforestation or renewable energy projects, to compensate for its remaining emissions. By taking these steps, your company can play a role in mitigating climate change and contributing to a more sustainable future.


E0507 16:52:24.543482000 13732294656 ssl_transport_security.cc:1511]   Handshake failed with fatal error SSL_ERROR_SSL: error:1000007d:SSL routines:OPENSSL_internal:CERTIFICATE_VERIFY_FAILED.


Question: What can my company do to help fight climate change?
There are many ways your company can help fight climate change. Here are some suggestions:

1. Reduce energy consumption: Encourage employees to turn off lights, computers, and other electronic devices when not in use. Install energy-efficient lighting and appliances, and consider using renewable energy sources like solar or wind power.
2. Implement sustainable practices: Encourage employees to use public transportation, carpool, or bike to work. Offer incentives for employees who use eco-friendly commuting methods.
3. Reduce waste: Encourage employees to reduce, reuse, and recycle. Implement a recycling program and composting program for food waste.
4. Use eco-friendly products: Switch to eco-friendly cleaning products, office supplies, and other products.
5. Support renewable energy projects: Consider investing in renewable energy projects, such as wind or solar farms, to help reduce your company's carbon footprint.
6. En

E0507 16:54:01.545698000 13840625664 ssl_transport_security.cc:1511]   Handshake failed with fatal error SSL_ERROR_SSL: error:1000007d:SSL routines:OPENSSL_internal:CERTIFICATE_VERIFY_FAILED.
E0507 16:54:07.733571000 13732294656 ssl_transport_security.cc:1511]   Handshake failed with fatal error SSL_ERROR_SSL: error:1000007d:SSL routines:OPENSSL_internal:CERTIFICATE_VERIFY_FAILED.
E0507 16:55:59.005150000 13732294656 ssl_transport_security.cc:1511]   Handshake failed with fatal error SSL_ERROR_SSL: error:1000007d:SSL routines:OPENSSL_internal:CERTIFICATE_VERIFY_FAILED.
E0507 16:56:08.368578000 13840625664 ssl_transport_security.cc:1511]   Handshake failed with fatal error SSL_ERROR_SSL: error:1000007d:SSL routines:OPENSSL_internal:CERTIFICATE_VERIFY_FAILED.
E0507 16:57:38.307764000 13732294656 ssl_transport_security.cc:1511]   Handshake failed with fatal error SSL_ERROR_SSL: error:1000007d:SSL routines:OPENSSL_internal:CERTIFICATE_VERIFY_FAILED.
E0507 16:58:00.818838000 13840625664 ssl